In [1]:
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.applications import inception_v3
from IPython import display
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import librosa
from pathlib import Path
import os, shutil
import random
plt.ioff()

In [2]:
#!pip install audiomentations
#!pip install soundfile

In [3]:
#!rm -rf data_augm
#!mkdir data_augm
#!mkdir data_augm/dataset

In [5]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

file_id = '1jwIWW2vuBJVO-XcCTL9HgmcolCfQJ2ir'    
download_file_from_google_drive(file_id, "data.zip")

In [6]:
from zipfile import ZipFile
if not os.path.exists('data'):
    os.makedirs('data')
zf = ZipFile('data.zip', 'r')
zf.extractall('data')
zf.close()

In [4]:
data_dir = pathlib.Path("data/data_final")

In [5]:
def decode_audio(audio_binary):
  audio, _ = tf.audio.decode_wav(audio_binary)
  return tf.squeeze(audio, axis=-1)

In [6]:
def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)

  # Note: You'll use indexing here instead of tuple unpacking to enable this 
  # to work in a TensorFlow graph.
  return parts[-2].numpy().decode()

In [7]:
def get_waveform(file_path):
  audio_binary = tf.io.read_file(file_path)
  waveform = decode_audio(audio_binary)
  return waveform, audio_binary

In [8]:
frame_length = 2048
frame_step = 512
num_mel_bins = 75
num_spectrogram_bins = (frame_length // 2) + 1
fmin = 0.0
sample_rate = 44100
fmax = sample_rate / 2


def get_spectrogram(waveform):
    # Padding for files with less than 16000 samples
    #zero_padding = tf.zeros([140000] - tf.shape(waveform), dtype=tf.float32) # NON SUPERARE I 3 SECONDI CON TIME STRETCH
    # Concatenate audio with padding so that all audio clips will be of the 
    # same length
    waveform = tf.cast(waveform, tf.float32)
    equal_length = waveform
    magnitude_spectrograms  = tf.signal.stft(
      equal_length, frame_length, frame_step)
    magnitude_spectrograms  = tf.abs(magnitude_spectrograms)
    
    # Step: magnitude_spectrograms->mel_spectrograms
    # Warp the linear-scale, magnitude spectrograms into the mel-scale.
    num_spectrogram_bins = magnitude_spectrograms.shape[-1]


    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins, num_spectrogram_bins, sample_rate, fmin,
        fmax)

    mel_spectrograms = tf.tensordot(
        magnitude_spectrograms, linear_to_mel_weight_matrix, 1)

    mel_spectrograms.set_shape(magnitude_spectrograms.shape[:-1].concatenate(
  linear_to_mel_weight_matrix.shape[-1:]))

    # Compute a stabilized log to get log-magnitude mel-scale spectrograms.
    log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)

    # Compute MFCCs from log_mel_spectrograms and take the first 13.
    #mfccs = tf.signal.mfccs_from_log_mel_spectrograms(
    #  log_mel_spectrograms)[..., :75]
  
    return log_mel_spectrograms

In [9]:
# VERSIONE PROSTAZIA
#def plot_spectrogram(spectrogram, label, class_path, i):
#  fig, ax = plt.subplots(figsize=(20,20))
#  mfcc_data= np.swapaxes(spectrogram, 0 ,1)
#  cax = ax.imshow(mfcc_data, interpolation='nearest', cmap=cm.coolwarm, origin='lower')
#  ax.axis("off")
#  fig.savefig('data\dataset\mels{}\mel_{}_{}.png'.format(class_path.split("data\dataset\data_final")[1], label, i), bbox_inches='tight', pad_inches=0, dpi=300)

In [10]:
# VERSIONE FEDOUS
def draw_spectrogram(spectrogram, output_dir_path, i):
    fig, ax = plt.subplots()
    fig.set_size_inches(2.24, 2.24)
    mfcc_data= np.swapaxes(spectrogram, 0 ,1)
    cax = ax.imshow(mfcc_data, interpolation='nearest', cmap=cm.coolwarm, origin='lower')
    ax.axis("off")
    fig.savefig(f'{output_dir_path}/mel_{i}.png', bbox_inches='tight', pad_inches=0, dpi=100)
    plt.close()
  

### Split in Train - Val - Test

In [14]:
#!pip install split-folders

In [15]:
import splitfolders
splitfolders.ratio("data/data_final", output="data/split", seed=1337, ratio=(.6, .2, .2))

### Data Augmentation

In [11]:
from audiomentations import Compose, AddGaussianNoise, PitchShift, Normalize, FrequencyMask
import numpy as np
import soundfile as sf
import librosa

In [17]:
augm_01 = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=1)
])

augm_02 = Compose([
    PitchShift(min_semitones=-4, max_semitones=4, p=1)
])

augm_03 = Compose([
    Normalize(p=1)
])

aug_l = [augm_01, augm_02, augm_03]

In [12]:
def augmentation(path, augment):
    y, sr = librosa.load(path)
    augmented_samples = augment(samples=y, sample_rate=sr)
    return augmented_samples, sr

In [19]:
main_dirs = ["data/split/train"] #2 folders

for main_dir in main_dirs:
    print(f"AUGMENTATION on {main_dir}")
    folders_pathlist = Path(main_dir).glob('*')

    for path in folders_pathlist: # per ogni cartella nella lista
        # because path is object not string
        class_path = str(path)
        print(class_path)
        wav_pathlist = Path(class_path).glob('./' + ('[0-9]' * 2) + ".wav")
        i = 300
        for w_path in wav_pathlist: # per ogni file audio
            #print(w_path)
            wav_path = str(w_path)
            label = get_label(wav_path)
            for aug in aug_l: # per ogni data augmentation da applicare
                augmented_samples, sr = augmentation(wav_path, aug)
                #print('{}/{}/{}.wav'.format(label,i)) #300 in avanti
                sf.write('{}/{}/{}.wav'.format(main_dir,label,i), augmented_samples, sr)
                i +=1


AUGMENTATION on data/split/train
data\split\train\Alces_alces
data\split\train\Bos_taurus
data\split\train\Cervus_elaphus
data\split\train\Equus_caballus
data\split\train\Lutra_lutra
data\split\train\Ovis
data\split\train\Pan
data\split\train\Panthera_leo
data\split\train\Procyon
data\split\train\Vulpes


In [20]:

#try:
#    os.makedirs('data/dataset/mels')
#except:
#    print("Folder already exists, deleting its content to replace them with new ones.")
#    shutil.rmtree('data/dataset/mels')
#    os.makedirs('data/dataset/mels')

### Wav -> MEL

In [13]:
#Define function to apply SpecAugment to wav files

def spec_augment(spec: np.ndarray, num_mask=1, 
                 freq_masking_max_percentage=0.15, time_masking_max_percentage=0.05):

    spec = spec.copy()
    for i in range(num_mask):
        all_frames_num, all_freqs_num = spec.shape
        freq_percentage = random.uniform(0.05, freq_masking_max_percentage)
        
        num_freqs_to_mask = int(freq_percentage * all_freqs_num)
        f0 = np.random.uniform(low=0.02, high=all_freqs_num - num_freqs_to_mask)
        f0 = int(f0)
        #Set band to 0
        spec[:, f0:f0 + num_freqs_to_mask] = 0

        time_percentage = random.uniform(0.0, time_masking_max_percentage)
        
        num_frames_to_mask = int(time_percentage * all_frames_num)
        t0 = np.random.uniform(low=0.0, high=all_frames_num - num_frames_to_mask)
        t0 = int(t0)
        #Set band to 0
        spec[t0:t0 + num_frames_to_mask, :] = 0
    
    return spec

In [14]:
input_dirs = ["data/split/train", "data/split/val", "data/split/test"] 
output_dirs = ["data_mel/train", "data_mel/val", "data_mel/test"] 

if not os.path.exists('data_mel'):
    os.makedirs('data_mel')
for output_dir in output_dirs:
  if not os.path.exists(output_dir):
    os.makedirs(output_dir) 

In [15]:
# VERSIONE FEDOUS
plt.ioff()
for i in range(3): # per le 3 cartelle train-val-test
    folds = os.listdir(f'{input_dirs[i]}/') # cartelle delle classi
  
    for fold in folds:
        if not os.path.exists(f'{output_dirs[i]}/{fold}/'):
            os.makedirs(f'{output_dirs[i]}/{fold}')
        wav_files = (os.listdir(f'{input_dirs[i]}/{fold}/'))
    
        j = 0

        for wav_file in wav_files: # per ogni file wav
            wav_path = f'{input_dirs[i]}/{fold}/{wav_file}'
            output_dir_path = f'{output_dirs[i]}/{fold}'
            wave, _ = get_waveform(wav_path)
            #label = get_label(wav_path)
            mel = get_spectrogram(wave)
            draw_spectrogram(mel, output_dir_path, j)
            j += 1
            #we now apply SpecAugment for the same wav if it is an original audio
            if len(wav_path.split("/")[-1]) == 6:
                mel = get_spectrogram(wave)
                warped_masked_spectrogram = spec_augment(mel.numpy())
                draw_spectrogram(warped_masked_spectrogram, output_dir_path, j)
                j += 1

        print(f'{output_dirs[i]}/{fold}/ DONE')

data_mel/train/Alces_alces/ DONE
data_mel/train/Bos_taurus/ DONE
data_mel/train/Cervus_elaphus/ DONE
data_mel/train/Equus_caballus/ DONE
data_mel/train/Lutra_lutra/ DONE
data_mel/train/Ovis/ DONE
data_mel/train/Pan/ DONE
data_mel/train/Panthera_leo/ DONE
data_mel/train/Procyon/ DONE
data_mel/train/Vulpes/ DONE
data_mel/val/Alces_alces/ DONE
data_mel/val/Bos_taurus/ DONE
data_mel/val/Cervus_elaphus/ DONE
data_mel/val/Equus_caballus/ DONE
data_mel/val/Lutra_lutra/ DONE
data_mel/val/Ovis/ DONE
data_mel/val/Pan/ DONE
data_mel/val/Panthera_leo/ DONE
data_mel/val/Procyon/ DONE
data_mel/val/Vulpes/ DONE
data_mel/test/Alces_alces/ DONE
data_mel/test/Bos_taurus/ DONE
data_mel/test/Cervus_elaphus/ DONE
data_mel/test/Equus_caballus/ DONE
data_mel/test/Lutra_lutra/ DONE
data_mel/test/Ovis/ DONE
data_mel/test/Pan/ DONE
data_mel/test/Panthera_leo/ DONE
data_mel/test/Procyon/ DONE
data_mel/test/Vulpes/ DONE
